# Machine Learning for Finance - Homework 2


In [47]:
import pandas as pd
import numpy as np
import pickle
import altair as alt

alt.data_transformers.disable_max_rows()

import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.linalg import cholesky
import scipy.linalg as la
import yfinance as yfin  
from pandas_datareader import data as pdr
from numpy.linalg import solve, norm
from statsmodels.stats.correlation_tools import cov_nearest

seed = 4

# Exercise 1

# Exercise 2

The stocks chosen for analysis were 'AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'FB', 'GOOG', 'HAL', 'HSBC'.

The dataframe is from 2015-01-01 to 2016-12-31.

In [48]:
# define functions 

def log_return(df):
    """Given dataframe, return its first order log return"""
    return np.log(df).diff().drop(index=df.index[0], axis=0, inplace=False)


def na_fill(df):
    # Simulates `na.fill(as.xts(rowAvgs(df),order.by=index(df)),"extend")`
    return df.interpolate(method="linear").fillna(method="bfill")


def create_index(df):
    # the linear method already does ffill for the tail values
    return na_fill(df.mean(axis=1, skipna=False))

def is_pos_def(x, epsilon):
    return np.all(np.linalg.eigvals(x) > epsilon)

In [49]:
# open dataset
with open('dataset.pkl', 'rb') as file:
    dataset = pd.read_pickle(file)

# define begin and end date
begin_date = "2014-12-31"
end_date = "2016-12-31"
prices = dataset["adjusted"].truncate(
    before=begin_date,
    after=end_date
)

# Select 9 stocks
stocks_namelist = ['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'FB', 'GOOG', 'HAL', 'HSBC']
prices = prices[stocks_namelist]

# transform prices to log prices
X = log_return(prices)
T, N = X.shape  

# plot normalized prices
alt.Chart((prices/prices.iloc[0,]).reset_index().melt(id_vars='index', value_vars=stocks_namelist)).mark_line().encode(
    x = alt.X('index', title=''),
    y = 'value',
    color = 'variable'
).properties(
    title='Normalized prices',
    height=300,
    width=800
)

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [50]:
# Split data into Training and Test data
T_trn = round(0.5*T)
X_trn = X.iloc[0:T_trn, ]
X_tst = X.iloc[T_trn:T, ]


# Function to calculate Robust (ellipsoid) Global Maximum Return Portfolio
def portfolioMaxReturnRobustEllipsoid(mu_hat, S, kappa=0.1):
    mu_hat = np.array(mu_hat)
    S12 = la.cholesky(S)  # S12.T @ S12 = Sigma
    w = cp.Variable(len(mu_hat))
    prob = cp.Problem(cp.Maximize(mu_hat @ w - kappa * cp.norm(S12 @ w, p=2)), constraints=[w >= 0, sum(w) == 1])
    result = prob.solve()
    return w.value

# Sentiment indicator PNlog factor model

In [51]:
PNlogMkt = create_index(dataset["PNlog"])  # PNlog Market index
SentIndx = PNlogMkt.loc[X.index]           # Sentiment inde
# Ensure data alignment
SentIndx = na_fill(SentIndx)

/var/folders/n4/wdz2gxc13wl6m46by5g242kh0000gn/T/ipykernel_12514/2085712792.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.interpolate(method="linear").fillna(method="bfill")


In [52]:
# Use Sentiment index for training
F_PNlog_trn = SentIndx.iloc[0:T_trn].to_frame()  # Convert series to dataframe

# Add a constant column to the factors for the intercept term
F_PNlog_trn.insert(0, 'ones', 1)

# Fit the model: X_trn ~ alpha + beta * SentIndx
Gamma_PNlog = pd.DataFrame(solve(F_PNlog_trn.T @ F_PNlog_trn, F_PNlog_trn.T.to_numpy() @ X_trn.to_numpy()).T, columns=["alpha", "beta"])
alpha_PNlog = Gamma_PNlog["alpha"]
B_PNlog = Gamma_PNlog[["beta"]]
E_PNlog = pd.DataFrame((X_trn.T - Gamma_PNlog.to_numpy() @ F_PNlog_trn.to_numpy().T).T, index=X_trn.index)
Psi_PNlog = (E_PNlog.T @ E_PNlog) / (T_trn - 2)


In [53]:
# Ensure F_PNlog_trn only includes the factors for the sentiment model (including the intercept)
F_PNlog_trn = F_PNlog_trn.drop(columns=['ones'])  

# Compute the covariance matrix of the factor returns
cov_factors = F_PNlog_trn.cov()

# Compute Sigma_PNlog
Sigma_PNlog = B_PNlog.to_numpy() @ cov_factors.to_numpy() @ B_PNlog.to_numpy().T + np.diag(np.diag(Psi_PNlog))


## K=0.95

In [54]:
kappa = 0.95

mu = X.mean(axis=0)
GMRP_robust_PNlog = portfolioMaxReturnRobustEllipsoid(mu, Sigma_PNlog, kappa)
GMRP_robust_PNlog_df = pd.DataFrame(GMRP_robust_PNlog)
GMRP_robust_PNlog_df['stock'] = stocks_namelist
GMRP_robust_PNlog_df.columns = ['w', 'stock']

# Plot the results
chart_095_1 = alt.Chart(GMRP_robust_PNlog_df).mark_bar(color='blue').encode(
    x='stock:N',
    y='w:Q'
)




In [55]:
# Robust Noisy Solution
w_all_GMRP_robust_ellipsoid = np.expand_dims(GMRP_robust_PNlog, axis=1)

np.random.seed(100)
for i in range(6):
    X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma_PNlog, size=T))
    mu_noisy = X_noisy.mean(axis=0)
    Sigma_noisy = X_noisy.cov()
    if not is_pos_def(Sigma_noisy, 0.001):
      Sigma_noisy = cov_nearest(Sigma_noisy)
    w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
    w_all_GMRP_robust_ellipsoid = np.concatenate((w_all_GMRP_robust_ellipsoid,
                                                   np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
w_all_GMRP_robust_ellipsoid["stock"] = stocks_namelist

long_w_all_GMRP_robust_ellipsoid = pd.melt(
    w_all_GMRP_robust_ellipsoid,
    id_vars=['stock'],
    var_name='run',
    value_name='weight')

chart_095_2 = alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
    x=alt.X('run:O', axis=None),
    y='weight:Q',
    column=alt.Column('stock:N', spacing=2)
).properties(width=125, title='Portfolilo weight by stock and noisy run')

In [56]:
chart_095_3=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar().encode(
    x=alt.X('run:O', axis=None),
    y=alt.Y('weight:Q', title='% weight'),
    color='stock:N',
    order=alt.Order('stock', sort='ascending')
).properties(width=500, title='Proportion of portfolio weight by stock')

## K=0.7

In [57]:
kappa = 0.7

GMRP_robust_PNlog = portfolioMaxReturnRobustEllipsoid(mu, Sigma_PNlog, kappa)
GMRP_robust_PNlog_df = pd.DataFrame(GMRP_robust_PNlog)
GMRP_robust_PNlog_df['stock'] = stocks_namelist
GMRP_robust_PNlog_df.columns = ['w', 'stock']

# Plot the results
chart_07_1=alt.Chart(GMRP_robust_PNlog_df).mark_bar(color='blue').encode(
    x='stock:N',
    y='w:Q'
)

In [58]:
# Robust Noisy Solution
w_all_GMRP_robust_ellipsoid = np.expand_dims(GMRP_robust_PNlog, axis=1)

np.random.seed(100)
for i in range(6):
    X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma_PNlog, size=T))
    mu_noisy = X_noisy.mean(axis=0)
    Sigma_noisy = X_noisy.cov()
    if not is_pos_def(Sigma_noisy, 0.001):
      Sigma_noisy = cov_nearest(Sigma_noisy)
    w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
    w_all_GMRP_robust_ellipsoid = np.concatenate((w_all_GMRP_robust_ellipsoid,
                                                   np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
w_all_GMRP_robust_ellipsoid["stock"] = stocks_namelist

long_w_all_GMRP_robust_ellipsoid = pd.melt(
    w_all_GMRP_robust_ellipsoid,
    id_vars=['stock'],
    var_name='run',
    value_name='weight')

chart_07_2=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
    x=alt.X('run:O', axis=None),
    y='weight:Q',
    column=alt.Column('stock:N', spacing=2)
).properties(width=125, title='Portfolilo weight by stock and noisy run')

In [59]:
chart_07_3=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar().encode(
    x=alt.X('run:O', axis=None),
    y=alt.Y('weight:Q', title='% weight'),
    color='stock:N',
    order=alt.Order('stock', sort='ascending')
).properties(width=500, title='Proportion of portfolio weight by stock')

## K=0.35

In [60]:
kappa = 0.35

GMRP_robust_PNlog = portfolioMaxReturnRobustEllipsoid(mu, Sigma_PNlog, kappa)
GMRP_robust_PNlog_df = pd.DataFrame(GMRP_robust_PNlog)
GMRP_robust_PNlog_df['stock'] = stocks_namelist
GMRP_robust_PNlog_df.columns = ['w', 'stock']

# Plot the results
chart_035_1=alt.Chart(GMRP_robust_PNlog_df).mark_bar(color='blue').encode(
    x='stock:N',
    y='w:Q'
)

In [61]:
# Robust Noisy Solution
w_all_GMRP_robust_ellipsoid = np.expand_dims(GMRP_robust_PNlog, axis=1)

np.random.seed(100)
for i in range(6):
    X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma_PNlog, size=T))
    mu_noisy = X_noisy.mean(axis=0)
    Sigma_noisy = X_noisy.cov()
    if not is_pos_def(Sigma_noisy, 0.001):
      Sigma_noisy = cov_nearest(Sigma_noisy)
    w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
    w_all_GMRP_robust_ellipsoid = np.concatenate((w_all_GMRP_robust_ellipsoid,
                                                   np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
w_all_GMRP_robust_ellipsoid["stock"] = stocks_namelist

long_w_all_GMRP_robust_ellipsoid = pd.melt(
    w_all_GMRP_robust_ellipsoid,
    id_vars=['stock'],
    var_name='run',
    value_name='weight')

chart_035_2=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
    x=alt.X('run:O', axis=None),
    y='weight:Q',
    column=alt.Column('stock:N', spacing=2)
).properties(width=125, title='Portfolilo weight by stock and noisy run')

In [62]:
chart_035_3=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar().encode(
    x=alt.X('run:O', axis=None),
    y=alt.Y('weight:Q', title='% weight'),
    color='stock:N',
    order=alt.Order('stock', sort='ascending')
).properties(width=500, title='Proportion of portfolio weight by stock')

## K=0.1

In [63]:
kappa = 0.1

GMRP_robust_PNlog = portfolioMaxReturnRobustEllipsoid(mu, Sigma_PNlog, kappa)
GMRP_robust_PNlog_df = pd.DataFrame(GMRP_robust_PNlog)
GMRP_robust_PNlog_df['stock'] = stocks_namelist
GMRP_robust_PNlog_df.columns = ['w', 'stock']

# Plot the results
chart_01_1=alt.Chart(GMRP_robust_PNlog_df).mark_bar(color='blue').encode(
    x='stock:N',
    y='w:Q'
)

In [64]:
# Robust Noisy Solution
w_all_GMRP_robust_ellipsoid = np.expand_dims(GMRP_robust_PNlog, axis=1)

np.random.seed(100)
for i in range(6):
    X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma_PNlog, size=T))
    mu_noisy = X_noisy.mean(axis=0)
    Sigma_noisy = X_noisy.cov()
    if not is_pos_def(Sigma_noisy, 0.001):
      Sigma_noisy = cov_nearest(Sigma_noisy)
    w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
    w_all_GMRP_robust_ellipsoid = np.concatenate((w_all_GMRP_robust_ellipsoid,
                                                   np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
w_all_GMRP_robust_ellipsoid["stock"] = stocks_namelist

long_w_all_GMRP_robust_ellipsoid = pd.melt(
    w_all_GMRP_robust_ellipsoid,
    id_vars=['stock'],
    var_name='run',
    value_name='weight')

chart_01_2=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
    x=alt.X('run:O', axis=None),
    y='weight:Q',
    column=alt.Column('stock:N', spacing=2)
).properties(width=125, title='Portfolilo weight by stock and noisy run')

In [65]:
chart_01_3=alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar().encode(
    x=alt.X('run:O', axis=None),
    y=alt.Y('weight:Q', title='% weight'),
    color='stock:N',
    order=alt.Order('stock', sort='ascending')
).properties(width=500, title='Proportion of portfolio weight by stock')

## Visualization and comparison

In [66]:
# Assuming you have another chart stored in a variable named 'chart2'
combined_chart_1 = alt.hconcat(chart_095_1, chart_07_1, chart_035_1, chart_01_1)
combined_chart_1

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/c

alt.HConcatChart(...)

In [69]:
# Horizontal combination of the first two charts
top_row_2 = alt.hconcat(chart_095_2, chart_07_2)

# Horizontal combination of the second two charts
bottom_row_2 = alt.hconcat(chart_035_2, chart_01_2)

# Vertical combination of the two rows
combined_chart_2 = alt.vconcat(top_row_2, bottom_row_2)

# Display the combined chart
combined_chart_2

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/c

alt.VConcatChart(...)

In [70]:
# Horizontal combination of the first two charts for the top row
top_row_3 = alt.hconcat(chart_095_3, chart_07_3)

# Horizontal combination of the second two charts for the bottom row
bottom_row_3 = alt.hconcat(chart_035_3, chart_01_3)

# Vertical combination of the two rows
combined_chart_3 = alt.vconcat(top_row_3, bottom_row_3)

# Display the combined chart
combined_chart_3


/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/c

alt.VConcatChart(...)